In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
import os
os.environ['USER_AGENT'] = 'dd'

# bs4_strainer = bs4.SoupStrainer(class_=("content-area"))
# loader = WebBaseLoader(
#     web_paths=("https://www.sdsc.edu/systems/expanse/user_guide.html",),
#     bs_kwargs={"parse_only": bs4_strainer},
# )

loader = WebBaseLoader("https://www.sdsc.edu/systems/expanse/index.html")
docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [3]:
# %pip install -qU langchain-ollama
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

local_embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [4]:

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke(question)

context = ' '.join([doc.page_content for doc in retrieved_docs])

In [5]:
context

'Expanse\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSystems\n\n\n\n\n\nFrom the earliest Cray systems to today’s high performance AI-integrated computing environments, SDSC enables discovery by architecting and operating innovative supercomputers; delivering research computing services and support; and providing infrastructure services including cloud, network, storage and data center colocation facilities.\n\n\n\nExpanse\nNational Research Platform\nSDSC Cloud\nSherlock Secure Cloud\nTriton Shared Computing Cluster\nVoyager\nCosmos\n\n\n\n\n\n\n\nServices\n\n\n\n\n\nFrom AI-enabled data science through end-to-end cyberinfrastructure solutions, SDSC’s experts provide academic researchers and industry partners with the consulting and training to take their projects to the next level.\n\n\n\nCloud\nColocation\nConsulting\nData Management\nHigh-Performance Computing\nRegulatory Compliance\nScience Gateways\nSoftware & Applications\nStorage\nWeb & Database Hosti

In [20]:
from langchain_ollama import OllamaLLM
question = "how many standard compute nodes are at SDSC?"
llm = OllamaLLM(model="mixtral:8x7b")
response = llm.invoke(f"""Answer the question according with a single number:
           Question: {question}.
           Context: {context}
""")

In [21]:
print(response)

728 standard compute nodes are at SDSC's Expanse.
